In [ ]:
import csv
import numpy as np

In [ ]:
# read data from csv into a numpy matrix
meteo_data = []

with open("meteo1.csv", "r") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=" ")
    for row in csv_reader:
        meteo_data.append(row)

meteo_data = np.asarray(meteo_data).astype(int)
print(meteo_data)

[[1 2 2 ... 0 0 2]
 [1 2 1 ... 1 2 1]
 [1 2 1 ... 2 1 2]
 ...
 [2 0 1 ... 1 2 0]
 [0 2 0 ... 0 2 2]
 [0 1 1 ... 0 1 1]]


In [ ]:
meteo_data.shape

(500, 100)

We have 500 realisations of Markov Chains of length 100.

In [ ]:
N = meteo_data.shape[0] # 500
M = meteo_data.shape[1] # 100
H = 3 # number of classes
S = 3 # number of states

def em_algorithm(init_strat="random", num_iter=100):

  if init_strat == "random":
    eta = np.random.rand(H)
    eta /= np.sum(eta)

    pi = np.random.rand(S,H)
    pi /= np.sum(pi, axis=0)

    A = np.random.rand(S,S,H)
    A /= np.sum(A, axis=1)

  elif init_strat == "uniform":
    eta = np.repeat(1/H,H)                # class probabilities (H)
    pi = np.repeat(1/S,H*S).reshape(S,H)    # initial distributions of the Markov Chains; (S,H)
    A = np.repeat(1/S,H*S*S).reshape(S,S,H) # transition probabilities of the Markov Chains (S,S,H)

  else:
    return None


  for iter in range(num_iter):

    # E step

    # q update (N,H)
    q = eta * pi[meteo_data[:, 0], :] * np.prod(A[meteo_data[:, :-1], meteo_data[:, 1:], :], axis=1)
    q = q / q.sum(axis=1, keepdims=True)


    # M step

    # eta update
    eta = q.sum(axis=0) / N

    # pi update
    for s in range(S):
      pi[s, :] = np.sum(q[meteo_data[:, 0] == s, :], axis=0) / np.sum(q, axis=0)

    # A update
    A = np.zeros_like(A)
    for n in range(N):
      for m in range(1, M):
        A[meteo_data[n, m-1], meteo_data[n, m], :] += q[n, :]
    A /= np.sum(A, axis=1, keepdims=True)


  # calculate log likelihood
  h_hat = np.argmax(q, axis=1)
  likelihood = eta[h_hat] * pi[meteo_data[:, 0], h_hat] * np.prod(A[meteo_data[:, :-1], meteo_data[:, 1:], h_hat[:, np.newaxis]], axis=1)
  log_likelihood = np.sum(np.log(likelihood))

  return h_hat, log_likelihood, q, eta, pi, A

h_hat, log_likelihood, q, eta, pi, A = em_algorithm(init_strat="random", num_iter=200)

In [ ]:
np.round(log_likelihood, 3)

-45255.667

In [ ]:
q[:10]

array([[2.06216772e-25, 1.48878824e-08, 9.99999985e-01],
       [1.00000000e+00, 2.30954113e-34, 2.26258282e-35],
       [1.00000000e+00, 3.50719071e-40, 2.65044484e-45],
       [1.00000000e+00, 3.84246130e-21, 1.75114283e-25],
       [1.00000000e+00, 1.85189472e-27, 3.63108812e-30],
       [1.00000000e+00, 3.90910876e-33, 1.46699703e-30],
       [3.54923783e-22, 6.09004656e-05, 9.99939100e-01],
       [1.00000000e+00, 6.87032044e-30, 2.08878031e-34],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 8.43241539e-35, 6.22525031e-37]])

In [ ]:
np.round(q[:10], 3)

array([[0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [ ]:
np.round(eta, 3)

array([0.808, 0.002, 0.19 ])

In [ ]:
np.round(pi[:,0], 3)

array([0.361, 0.364, 0.275])

In [ ]:
np.round(pi[:,1], 3)

array([0., 1., 0.])

In [ ]:
np.round(pi[:,2], 3)

array([0.432, 0.568, 0.   ])

In [ ]:
np.round(A[:,:,0], 3)

array([[0.067, 0.37 , 0.563],
       [0.134, 0.259, 0.606],
       [0.472, 0.433, 0.094]])

In [ ]:
np.round(A[:,:,1], 3)

array([[0.275, 0.325, 0.4  ],
       [0.331, 0.285, 0.384],
       [0.378, 0.021, 0.601]])

In [ ]:
np.round(A[:,:,2], 3)

array([[0.39 , 0.147, 0.463],
       [0.239, 0.518, 0.242],
       [0.547, 0.018, 0.435]])

In [ ]:
h_hat[:10]

array([2, 0, 0, 0, 0, 0, 2, 0, 0, 0])

In [ ]:
h_hat, log_likelihood, q, eta, pi, A = em_algorithm(init_strat="uniform", num_iter=200)

In [ ]:
eta

array([0.33333333, 0.33333333, 0.33333333])

In [ ]:
pi[:,0]

array([0.374, 0.404, 0.222])

In [ ]:
pi[:,1]

array([0.374, 0.404, 0.222])

In [ ]:
pi[:,2]

array([0.374, 0.404, 0.222])

In [ ]:
np.round(A[:,:,0], 3)

array([[0.16 , 0.306, 0.534],
       [0.145, 0.283, 0.572],
       [0.487, 0.35 , 0.163]])

In [ ]:
log_likelihood

-50022.85343371084

In [ ]:
q

array([[0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       ...,
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333]])